<a href="https://colab.research.google.com/github/olaviinha/NeuralImageColorization/blob/master/DeOldify_ImageColorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeOldify Image Colorization Batch Processer

This is a batch processor for [DeOldify](https://github.com/jantic/DeOldify) to be run in Google Colaboratory, using Google Drive as data storage.

### Tips
- `input` may be a file path or a directory path. If a directory path given, all images in the directory will be processed.
- if `output_dir` is not given, images will be saved to `<input_dir>/colorized`
- Set all direcotries relative to your Google Drive root, e.g. `images/grayscale` and `images/colorized` if you have a directory called _images_ in your Drive, containing directory _grayscale_.




In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.<br>
#@markdown <small>Mounting Drive will enable this notebook to save outputs directly to your Drive. Otherwise you will need to copy/download them manually from this notebook.</small>

force_setup = False
pip_packages = ''
main_repository = 'https://github.com/jantic/DeOldify.git'
mount_drive = True #@param {type:"boolean"}

# Download the repo from Github
import os
from google.colab import output
import warnings
warnings.filterwarnings('ignore')
%cd /content/

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  !pip -q install import-ipynb {pip_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if mount_drive is True:
  if not os.path.isdir('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')
    drive_root = '/content/drive/My Drive'
  if not os.path.isdir('/content/mydrive'):
    os.symlink('/content/drive/My Drive', '/content/mydrive')
    drive_root = '/content/mydrive/'
  drive_root_set = True
else:
  create_dirs(['/content/faux_drive'])
  drive_root = '/content/faux_drive/'

if main_repository is not '':
  !git clone {main_repository}

%cd DeOldify

#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
  print('GPU not available.')

!pip install -r requirements-colab.txt



import fastai
from deoldify.visualize import *
torch.backends.cudnn.benchmark = True

!mkdir 'models'
!wget https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth -O ./models/ColorizeArtistic_gen.pth
!wget https://www.dropbox.com/s/usf7uifrctqw9rl/ColorizeStable_gen.pth?dl=0 -O ./models/ColorizeStable_gen.pth

output.clear()
# !nvidia-smi
op(c.ok, 'Setup finished.')

In [ ]:
#@title #Colorize
input = "images/grayscale" #@param {type: "string"}
output_dir = "images/colorized" #@param {type: "string"}
useArtistic = False #@param {type:"boolean"}
render_factor = 35  #@param {type: "slider", min: 7, max: 40}

if os.path.isfile(drive_root+input):
  pictures = [drive_root+input]
  source_path = path_dir(drive_root+input)
elif os.path.isdir(drive_root+input):
  source_path = fix_path(drive_root+input)
  pictures = list_images(source_path)
  pictures.sort()
else:
  op(c.fail, 'Unable to find', input)


if output_dir == '':
  output_path = source_path+'colorized/'
  op(c.warn, 'No output_dir was given, saving to', output_path.replace(drive_root, ''))
  print()
else:
  output_path = fix_path(drive_root+output_dir)

if not os.path.isdir(output_path):
  os.mkdir(output_path)



count = len(pictures)

op(c.title, 'Coloring '+str(count)+' images.\n')
for i, pic in enumerate(pictures):
  index = i+1
  if pic is not None and pic !='':
    #pic_path = str(source_path)+"/"+pic
    pic_out = output_path+path_leaf(pic)
    if not os.path.isfile(pic_out):
      op(c.title, 'Coloring '+str(index)+'/'+str(count), path_leaf(pic))
      colorizer = get_image_colorizer(artistic=useArtistic)
      image_path = colorizer.plot_transformed_image(path=pic, render_factor=render_factor, compare=False, watermarked=False)
      im = Image.open(image_path)
      im.save(pic_out)
      del colorizer
    else:
      op(c.warn, 'Image already exists. Skipping', path_leaf(pic_out))
  else:
    op(c.fail, 'No images found, I guess.')